In [23]:
import pandas as pd
import openpyxl
import ifcopenshell
import ifcopenshell.util.element
from collections import Counter


In [24]:
df_matrix = pd.read_excel(r'C:\Users\lucas.galicioli\ifc-classifier\data\interim\classification-matrix.xlsx')
display(df_matrix)

,Class,PredefinedType,BuildingStorey,Material,Name,PSET_RÔGGA.RÔGGA_SEÇÃO,PSET_RÔGGA.RÔGGA_DESCRIÇÃO,Width,Thickness,Length,Height,FileName,Ô_CLS_DISCIPLINAS,Ô_CLS_CLASSIFICAÇÃO_SOLIBRI,Classification Name
0,IfcWall,STANDARD,NaN,*ALV*,NaN,NaN,NaN,NaN,"0,09<=X<=0,11",NaN,NaN,NaN,Arquitetura,*Parede*,70303001
1,IfcWall,STANDARD,NaN,*dry*R*U*,NaN,NaN,NaN,NaN,"0,059<=X<=0,061",NaN,NaN,NaN,Arquitetura,Drywall,704104
2,IfcWall,STANDARD,NaN,NaN,NaN,*PAINEL*ACM*10*,NaN,NaN,NaN,NaN,NaN,NaN,Arquitetura,*Parede*,1604102
3,IfcBuildingElementProxy,NOTDEFINED,NaN,NaN,NaN,*CHURR*COB*G5*,NaN,NaN,NaN,NaN,NaN,NaN,Arquitetura,Churrasqueira pré-fabricada,80203105
4,IfcBuildingElementProxy,NOTDEFINED,NaN,NaN,NaN,*CHURR*COB*G6*,NaN,NaN,NaN,NaN,NaN,NaN,Arquitetura,Churrasqueira pré-fabricada,80203106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7096,IfcSanitaryTerminal,FLOORWASTE,NaN,NaN,NaN,*RAL*LIN*PVC*,NaN,NaN,NaN,NaN,"0,68>=X>=0,72",NaN,Sanitário,NaN,170303123
7097,IfcSanitaryTerminal,FLOORWASTE,NaN,NaN,NaN,*RAL*LIN*PVC*,NaN,NaN,NaN,NaN,"0,48>=X>=0,52",NaN,Sanitário,NaN,170303124
7098,IfcWasteTerminal,NaN,NaN,NaN,NaN,*EST*ELE*ESG*,NaN,NaN,NaN,NaN,NaN,NaN,Sanitário,NaN,1708104
7099,IfcWasteTerminal,NaN,NaN,NaN,NaN,*EST*ELE*ESG*,NaN,NaN,NaN,NaN,NaN,NaN,Sanitário,NaN,1708106


In [25]:
# model dataset
ifc_file_doc = r'C:\Users\lucas.galicioli\ifc-classifier\data\raw\PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00.ifc'

ifc_file = ifcopenshell.open(ifc_file_doc)

# Define dataset
element_data = []

products = ifc_file.by_type('IfcProduct')

for product in products:
    psets = ifcopenshell.util.element.get_psets(product)

    element_info = {
        'GlobalId' : product.GlobalId,
        'Class' : product.is_a(),
        'Name' : getattr(product, 'Name', None)
    }

    for pset_name, properties in psets.items():
        for prop_name, prop_value in properties.items():
            column_name = f"{pset_name}.{prop_name}"
            element_info[column_name] = prop_value

    element_data.append(element_info)

df_ifc_data = pd.DataFrame(element_data).fillna(pd.NA)

# Na hora de exibir, use .fillna('') para substituir os valores ausentes por uma string vazia
display(df_ifc_data.head().fillna(''))

,GlobalId,Class,Name,Pset_EnvironmentalImpactIndicators.Reference,Pset_EnvironmentalImpactIndicators.id,Pset_ValveTypeCommon.Reference,Pset_ValveTypeCommon.id,PSET_RÔGGA.RÔGGA_DESCRIÇÃO,PSET_RÔGGA.RÔGGA_MACROLOG,PSET_RÔGGA.RÔGGA_EAP,...,Pset_BuildingElementProxyCommon.Reference,Pset_BuildingElementProxyCommon.IsExternal,Pset_BuildingElementProxyCommon.id,Pset_BuildingStoreyCommon.Reference,Pset_BuildingStoreyCommon.AboveGround,Pset_BuildingStoreyCommon.id,Pset_BuildingSystemCommon.Reference,Pset_BuildingSystemCommon.id,Pset_SiteCommon.Reference,Pset_SiteCommon.id
0,1n494Wk3P0mBiGCJTDJMTY,IfcValve,"Registro de Pressão:3/4"":2734253","Registro de Pressão:3/4""",3380.0,"Registro de Pressão:3/4""",3381.0,Registro de pressão,TOR1,TER,...,,,,,,,,,,
1,1n494Wk3P0mBiGCJTDJHze,IfcValve,"Registro de Pressão:3/4"":2736295","Registro de Pressão:3/4""",13445.0,"Registro de Pressão:3/4""",13446.0,Registro de pressão,TOR1,TER,...,,,,,,,,,,
2,1n494Wk3P0mBiGCJTDJHip,IfcValve,"Registro de Pressão:3/4"":2737404","Registro de Pressão:3/4""",21241.0,"Registro de Pressão:3/4""",21242.0,Registro de pressão,TOR1,TER,...,,,,,,,,,,
3,1n494Wk3P0mBiGCJTDJHfw,IfcValve,"Registro de Pressão:3/4"":2737589","Registro de Pressão:3/4""",22779.0,"Registro de Pressão:3/4""",22780.0,Registro de pressão,TOR1,TER,...,,,,,,,,,,
4,1n494Wk3P0mBiGCJTDJHU5,IfcValve,"Registro de Pressão:3/4"":2738250","Registro de Pressão:3/4""",24829.0,"Registro de Pressão:3/4""",24830.0,Registro de pressão,TOR1,TER,...,,,,,,,,,,


In [26]:
import ifcopenshell
import ifcopenshell.util.element
import pandas as pd
import os

# --- Caminho do arquivo IFC ---
ifc_file_path = r'C:\Users\lucas.galicioli\ifc-classifier\data\raw\PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00.ifc'

# --- Funções Auxiliares (com a função de quantidade modificada) ---

def get_building_storey(element):
    """ Encontra o IfcBuildingStorey no qual o elemento está contido. """
    try:
        spatial_container = ifcopenshell.util.element.get_container(element)
        if spatial_container and spatial_container.is_a('IfcBuildingStorey'):
            return spatial_container.Name
    except Exception:
        pass
    return None

def get_material_name(element):
    """ Extrai o nome do material associado ao elemento. """
    material = ifcopenshell.util.element.get_material(element)
    if not material:
        return None
    if hasattr(material, 'Name'):
        return material.Name
    elif hasattr(material, 'MaterialLayers'):
        layer_names = [
            layer.Material.Name 
            for layer in material.MaterialLayers 
            if hasattr(layer, 'Material') and hasattr(layer.Material, 'Name')
        ]
        return ', '.join(layer_names) if layer_names else None
    return None
    
# --- SOLUÇÃO 2: Função de quantidade compatível com versões antigas ---
def get_quantity_value_legacy(element, quantity_name):
    """
    Busca por uma quantidade específica (ex: 'Width') e retorna seu valor.
    Esta versão é compatível com versões mais antigas do ifcopenshell sem 'get_qsets'.
    """
    # Itera através das relações de definição do elemento
    for definition in getattr(element, 'IsDefinedBy', []):
        if definition.is_a('IfcRelDefinesByProperties'):
            prop_set = definition.RelatingPropertyDefinition
            # Verifica se é um conjunto de quantidades (IfcElementQuantity)
            if prop_set.is_a('IfcElementQuantity'):
                # Itera através das quantidades dentro do conjunto
                for quantity in prop_set.Quantities:
                    if quantity.Name == quantity_name:
                        # Extrai o valor do atributo correto (ex: LengthValue, AreaValue)
                        value_attribute = next((attr for attr in dir(quantity) if attr.endswith('Value')), None)
                        if value_attribute:
                            return getattr(quantity, value_attribute)
    return None

# --- Processamento Principal ---

try:
    ifc_file = ifcopenshell.open(ifc_file_path)
    file_name = os.path.basename(ifc_file_path)

    element_data = []
    products = ifc_file.by_type('IfcProduct')

    print(f"Processando {len(products)} elementos do arquivo: {file_name}...")

    for product in products:
        if product.is_a('IfcOpeningElement') or product.is_a('IfcVirtualElement'):
            continue

        psets = ifcopenshell.util.element.get_psets(product)
        rogga_pset = psets.get('PSET_RÔGGA', {})

        element_info = {
            'GlobalId': product.GlobalId,
            'FileName': file_name,
            'Class': product.is_a(),
            'PredefinedType': getattr(product, 'PredefinedType', None),
            'Name': getattr(product, 'Name', None),
            'BuildingStorey': get_building_storey(product),
            'Material': get_material_name(product),
            'PSET_RÔGGA.RÔGGA_SEÇÃO': rogga_pset.get('RÔGGA_SEÇÃO', None),
            'PSET_RÔGGA.RÔGGA_DESCRIÇÃO': rogga_pset.get('RÔGGA_DESCRIÇÃO', None),
            
            # ATENÇÃO: Usando a nova função 'legacy' aqui
            'Width': get_quantity_value_legacy(product, 'Width'),
            'Thickness': get_quantity_value_legacy(product, 'Thickness'),
            'Length': get_quantity_value_legacy(product, 'Length'),
            'Height': get_quantity_value_legacy(product, 'Height'),
        }
        
        element_data.append(element_info)

    df_ifc_data = pd.DataFrame(element_data)

    desired_order = [
        'Class', 'PredefinedType', 'BuildingStorey', 'Material', 'Name',
        'PSET_RÔGGA.RÔGGA_SEÇÃO', 'PSET_RÔGGA.RÔGGA_DESCRIÇÃO',
        'Width', 'Thickness', 'Length', 'Height',
        'FileName', 'GlobalId'
    ]

    for col in desired_order:
        if col not in df_ifc_data.columns:
            df_ifc_data[col] = None
            
    df_ifc_data = df_ifc_data[desired_order]

    print("\nDataset criado com sucesso! Amostra dos dados:")
    display(df_ifc_data.head().fillna(''))

except FileNotFoundError:
    print(f"ERRO: O arquivo não foi encontrado em: {ifc_file_path}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Processando 23424 elementos do arquivo: PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00.ifc...

Dataset criado com sucesso! Amostra dos dados:


,Class,PredefinedType,BuildingStorey,Material,Name,PSET_RÔGGA.RÔGGA_SEÇÃO,PSET_RÔGGA.RÔGGA_DESCRIÇÃO,Width,Thickness,Length,Height,FileName,GlobalId
0,IfcValve,NOTDEFINED,01. TER,,"Registro de Pressão:3/4"":2734253",ø 25 x ø 25,Registro de pressão,,,,,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJMTY
1,IfcValve,NOTDEFINED,01. TER,,"Registro de Pressão:3/4"":2736295",ø 25 x ø 25,Registro de pressão,,,,,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHze
2,IfcValve,NOTDEFINED,01. TER,,"Registro de Pressão:3/4"":2737404",ø 25 x ø 25,Registro de pressão,,,,,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHip
3,IfcValve,NOTDEFINED,01. TER,,"Registro de Pressão:3/4"":2737589",ø 25 x ø 25,Registro de pressão,,,,,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHfw
4,IfcValve,NOTDEFINED,01. TER,,"Registro de Pressão:3/4"":2738250",ø 25 x ø 25,Registro de pressão,,,,,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHU5


In [29]:
# Supondo que 'df_ifc_data' e 'df_matrix' já estão carregados e alinhados

# Passo 1: Defina as colunas que servem como chave para a correspondência
# (Use os nomes reais das suas colunas aqui)
colunas_chave = [
    'Class',
    'PredefinedType',
    'BuildingStorey',
    'Material',
    'Name',
    'PSET_RÔGGA.RÔGGA_SEÇÃO',
    'PSET_RÔGGA.RÔGGA_DESCRIÇÃO',
    'Width',
    'Thickness',
    'Length',
    'Height',
    'FileName'


]


for col in colunas_chave:
    df_ifc_data[col] = df_ifc_data[col].astype(str)
    df_matrix[col] = df_matrix[col].astype(str)

# Passo 2 e 3: Execute o merge e armazene o resultado
df_final = pd.merge(
    left=df_ifc_data,
    right=df_matrix,
    how='left',
    on=colunas_chave
)

# Passo Bônus: Verifique o resultado
print("Dimensões do DataFrame antes do merge:", df_ifc_data.shape)
print("Dimensões do DataFrame após o merge:", df_final.shape)
display(df_final)

Dimensões do DataFrame antes do merge: (23424, 13)
Dimensões do DataFrame após o merge: (23424, 16)


,Class,PredefinedType,BuildingStorey,Material,Name,PSET_RÔGGA.RÔGGA_SEÇÃO,PSET_RÔGGA.RÔGGA_DESCRIÇÃO,Width,Thickness,Length,Height,FileName,GlobalId,Ô_CLS_DISCIPLINAS,Ô_CLS_CLASSIFICAÇÃO_SOLIBRI,Classification Name
0,IfcValve,NOTDEFINED,01. TER,None,"Registro de Pressão:3/4"":2734253",ø 25 x ø 25,Registro de pressão,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJMTY,NaN,NaN,NaN
1,IfcValve,NOTDEFINED,01. TER,None,"Registro de Pressão:3/4"":2736295",ø 25 x ø 25,Registro de pressão,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHze,NaN,NaN,NaN
2,IfcValve,NOTDEFINED,01. TER,None,"Registro de Pressão:3/4"":2737404",ø 25 x ø 25,Registro de pressão,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHip,NaN,NaN,NaN
3,IfcValve,NOTDEFINED,01. TER,None,"Registro de Pressão:3/4"":2737589",ø 25 x ø 25,Registro de pressão,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHfw,NaN,NaN,NaN
4,IfcValve,NOTDEFINED,01. TER,None,"Registro de Pressão:3/4"":2738250",ø 25 x ø 25,Registro de pressão,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,1n494Wk3P0mBiGCJTDJHU5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23419,IfcBuildingStorey,None,None,None,11. TP1,None,None,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,2teNiaSXdKHH5wP_7Vdd$L,NaN,NaN,NaN
23420,IfcBuildingStorey,None,None,None,12. TP1,None,None,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,3nGKeXqOPtAlDPvgCrtF1B,NaN,NaN,NaN
23421,IfcBuildingStorey,None,None,None,13. COB,None,None,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,3jUP9nL$gdvmNYyl7ED7WB,NaN,NaN,NaN
23422,IfcSite,None,None,None,Default,None,None,None,None,nan,None,PHN21021A-HID-LO-0002-BIM-T01-GER-TORRE 01-R00...,2U1CLWFU10Df6XKZpVtEQm,NaN,NaN,NaN
